In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import math

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, LSTM, GRU

from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import MinMaxScaler
import datetime, os


pd.set_option('display.max_columns',30)
pd.set_option('display.max_rows',10)



In [ ]:
# load data
data = pd.read_csv('https://raw.githubusercontent.com/SkatAI/deeplearning/master/data/sunspots.csv')
data.columns = ['n', 'date', 'number']
data.index = data['date']
data.head()

In [ ]:
plt.plot(data['number'])

In [ ]:
# split train, test
training_data_len = math.ceil(len(data) * .8)
training_data_len

#Splitting the dataset
train_data = data[:training_data_len]['number']
test_data = data[training_data_len:]['number']
print(train_data.head())
print(train_data.shape, test_data.shape)

In [ ]:

dataset_train = train_data.values
dataset_train = np.reshape(dataset_train, (-1,1))
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(dataset_train)


In [ ]:
dataset_test = test_data.values
dataset_test = np.reshape(dataset_test, (-1,1))
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_test = scaler.fit_transform(dataset_test)


In [ ]:
# build sequences
sequence_length = 50

X_train = []
y_train = []
for i in range(50, len(scaled_train)):
	X_train.append(scaled_train[i-sequence_length:i, 0])
	y_train.append(scaled_train[i, 0])


X_test = []
y_test = []
for i in range(50, len(scaled_test)):
	X_test.append(scaled_test[i-sequence_length:i, 0])
	y_test.append(scaled_test[i, 0])

In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)

#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0],1))
print("X_train :",X_train.shape,"y_train :",y_train.shape)


In [ ]:
X_test, y_test = np.array(X_test), np.array(y_test)

#Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
print("X_test :",X_test.shape,"y_test :",y_test.shape)

In [ ]:
%load_ext tensorboard


# Modelisation

In [ ]:
%tensorboard --logdir logs


In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


In [ ]:

model = Sequential()
model.add(SimpleRNN(100, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(1, activation = 'linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')


model.summary()

In [ ]:
# Train the model
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


model.fit(X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_data = (X_test, y_test),
    callbacks=[tensorboard_callback]
)


In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test.flatten(), y_pred.flatten()))

print(f"rmse {rmse}")

In [ ]:
# 2 ieme modele
# regularization L2

model = Sequential()
# on remplace :
# model.add(SimpleRNN(100, activation='relu', input_shape=(sequence_length, 1)))

# par :
model.add(SimpleRNN(100, activation='relu', input_shape=(sequence_length, 1),
                    kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))


# model.add(Dropout(0.5))
model.add(Dense(1, activation = 'linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')


model.summary()

# tensorboard call back
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


model.fit(X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_data = (X_test, y_test),
    callbacks=[tensorboard_callback]
)


In [ ]:
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test.flatten(), y_pred.flatten()))

print(f"rmse {rmse}")

# Ensuite

continuer a ameliorer la perf
- cell : LSTM ou GRU
-


In [ ]:
model = Sequential()
# on remplace :
# model.add(SimpleRNN(100, activation='relu', input_shape=(sequence_length, 1)))

# par :
model.add(LSTM(100, activation='relu', input_shape=(sequence_length, 1),
                    kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))

# model.add(Dropout(0.5))
model.add(Dense(1, activation = 'linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')


model.summary()

# tensorboard call back
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


model.fit(X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_data = (X_test, y_test),
    callbacks=[tensorboard_callback]
)

In [ ]:
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test.flatten(), y_pred.flatten()))

print(f"rmse {rmse}")